**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
!pip install scikit-video # added
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten

     |████████████████████████████████| 2.3MB 2.8MB/s 


Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act is used to return the action to do according to a state as input and a probability parameterized by epsilon. The value of epsilon is essential in the trade-off Exploration-Exploitation. A high value will favour exploration i.e. explore the actions in order to learn about them and improves the chances of the agent of getting future reward. A small value will favour exploitation i.e. the agent uses what it has already learned to maximize its total reward. The agent exploits by taking the actions it expects to provide the largest rewards.



***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256 
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        # self.position[-2:, :] = -1 # rather self.position[:,-2:]
        self.position[:,-2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0: # right
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1: # left
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2: # down
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3: # up
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        # self.position[-2:, :] = -1 # rather self.position[:,-2:] 
        self.position[:,-2:] = -1

        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=21 # set small when debugging
epochs_test=11 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array `board` contains the rewards of each cell (0, +0.5 or -1), each cell visited by the rat changes the reward to 0.
`position` has 0 in all cells except the one the rat is in (with value 1) and the boarders (2 first lines and columns, 2 last lines and columns) with value -1. These two arrays are used in the states of the model by concatenation of the values for each of the cells in the visibility of the rat (5^2 cells)

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, self.n_action, size=1)[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE

        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()

        game_over = False
    
        win = 0
        lose = 0
    
        while not game_over:
            # The agent performs an action
            action = agent.act(state)
    
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)
    
            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward


        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [9]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 11.5/13.0. Average score (-1.5)
Win/lose count 10.5/14.0. Average score (-2.5)
Win/lose count 12.0/19.0. Average score (-4.0)
Win/lose count 9.5/11.0. Average score (-3.375)
Win/lose count 8.0/12.0. Average score (-3.5)
Win/lose count 13.5/10.0. Average score (-2.3333333333333335)
Win/lose count 4.0/15.0. Average score (-3.5714285714285716)
Win/lose count 8.0/14.0. Average score (-3.875)
Win/lose count 9.0/10.0. Average score (-3.5555555555555554)
Win/lose count 14.5/18.0. Average score (-3.55)
Win/lose count 6.5/11.0. Average score (-3.6363636363636362)
Final score: -3.6363636363636362


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




1) Let's prove that  

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')}[r(s,a,s')+\gamma Q^{\pi}(s',a')]
\end{equation*}

We have: 

\begin{align*}
Q^{\pi}(s,a) &= E_{p^{\pi}} [\sum_{t\geq0} \gamma^t r(s_t, a_t) | s_0=s, a_0=a] \\
&= r(s,a) + E_{p^{\pi}} [\sum_{t\geq1} \gamma^t r(s_t, a_t) | s_0=s, a_0=a] \\
&= r(s,a) + E_{(s',a')}[\gamma E_{p^{\pi}}[\sum_{t\geq1} \gamma^{t-1} r(s_t, a_t) | s_1=s', a_1=a']] \\
&= r(s,a) + E_{(s',a')}[\gamma E_{p^{\pi}}[\sum_{t\geq0} \gamma^{t} r(s_t, a_t) | s_0=s', a_0=a']] \\
&= r(s,a) + E_{(s',a')}[\gamma Q^{\pi}(s',a')] \\
&= E_{(s',a')}[r(s,a,s')+\gamma Q^{\pi}(s',a')]
\end{align*}


2) Let $\pi^*$ be the optimal policy. We want to prove that:

\begin{equation*}
Q^{*}(s,a)=E_{s'}[r(s,a,s')+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}

We have: 

\begin{align*}
Q^{*}(s,a)&= \max_{\pi} Q^{\pi}(s,a)  \\
&= \max_{\pi} E_{(s',a')}[r(s,a,s')+\gamma Q^{\pi}(s',a')]\\
&= \max_{\pi} (r(s,a) + E_{(s',a')}[\gamma Q^{\pi}(s',a')])\\
&= \max_{\pi} (r(s,a) + E_{s'}[\gamma \max_{a'}Q^{\pi}(s',a')])\\
&= r(s,a) + E_{s'}[\gamma \max_{\pi, a'} Q^{\pi}(s',a')]\\
&= r(s,a) + E_{s'}[\gamma  \max_{a'} Q^{*}(s',a')]\\
&= E_{s'}[r(s,a,s')+\gamma\max_{a'}Q^{*}(s',a')]
\end{align*}

3) The idea is to train a deep network $Q(s,a,\theta)$ of paramater $\theta$ to predict $Q^{*}(s,a)$ given observation $s$ and action $a$.
As we are in the context of a regression problem and we want to minimize the distance between the output of the network and $Q^{*}(s,a)$, one could use the loss function: 

\begin{equation*}
\mathcal{L}(\theta)=E_{s'}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*} 

using the result of the previous question

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
      if len(self.memory) < self.max_memory: # the maximum memory size is not reached thus the new move can be appended to the list
        self.memory.append(m)
      else:
        self.memory.pop(0) # the maximum memory size is reached thus one move (the first one) needs to be removed
        self.memory.append(m)

    def random_access(self):
        return self.memory[np.random.randint(low=0, high=len(self.memory))]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape(1,-1)))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            states = self.memory.random_access()
            input_states[i] = states[0]
            next_state = states[1]
            action = states[2]
            reward = states[3]
            game_over_= states[4]
            
            target_q[i] = self.model.predict(input_states[i].reshape(1,-1))[0]
            if game_over_:
                ######## FILL IN
                target_q[i][action] = reward
            else:
                ######## FILL IN
                target_q[i][action] = reward + self.discount * np.max(self.model.predict(next_state.reshape(1,-1)))

        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states.reshape(self.batch_size, 25*self.n_state), target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)favoured
        # model.compile("sgd", "mse")
        model.compile("adam", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        model = Sequential()
        model.add(Dense(50, activation='relu'))
        model.add(Dense(30, activation='relu'))
        model.add(Dense(30, activation='relu'))
        model.add(Dense(self.n_action, activation='linear'))
        
        # model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse") # test adam
        model.compile(loss='mse', optimizer=keras.optimizers.Adam(lr=0.001))
        self.model = model
        

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 128) # lr=0.1, memory_size=2000, bs = 32
train(agent, env, epochs_train, prefix='fc_train') # time execution : 19 minutes
HTML(display_videos('fc_train10.mp4')) 













Epoch 000/021 | Loss 0.0008 | Win/lose count 2.5/4.0 (-1.5)
Epoch 001/021 | Loss 0.0290 | Win/lose count 2.0/4.0 (-2.0)
Epoch 002/021 | Loss 0.0036 | Win/lose count 4.0/6.0 (-2.0)
Epoch 003/021 | Loss 0.0130 | Win/lose count 4.0/0 (4.0)
Epoch 004/021 | Loss 0.0065 | Win/lose count 3.0/2.0 (1.0)
Epoch 005/021 | Loss 0.0023 | Win/lose count 8.5/5.0 (3.5)
Epoch 006/021 | Loss 0.0020 | Win/lose count 4.0/6.0 (-2.0)
Epoch 007/021 | Loss 0.0015 | Win/lose count 4.5/2.0 (2.5)
Epoch 008/021 | Loss 0.0085 | Win/lose count 4.5/2.0 (2.5)
Epoch 009/021 | Loss 0.0033 | Win/lose count 5.0/6.0 (-1.0)
Epoch 010/021 | Loss 0.0044 | Win/lose count 3.0/0 (3.0)
Epoch 011/021 | Loss 0.0022 | Win/lose count 5.0/3.0 (2.0)
Epoch 012/021 | Loss 0.0024 | Win/lose count 1.5/3.0 (-1.5)
Epoch 013/021 | Loss 0.0141 | Win/lose count 11.0/4.0 (7.0)
Epoch 014/021 | Loss 0.0020 | Win/lose count 3.5/3.0 (0.5)
Epoch 015/021 | Loss 0.0107 | Win/lose count 12.5/5.0 (7.5)
Epoch 016/021 | Loss 0.0138 | Win/lose c

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Reshape(target_shape=(5,5,self.n_state)))
        model.add(Conv2D(filters=16, kernel_size=3, activation='relu', padding='same'))
        model.add(Conv2D(filters=16, kernel_size=3, activation='relu', padding='same'))
        model.add(Flatten())
        model.add(Dense(self.n_action, activation='linear'))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        # model.compile(loss='mse', optimizer=keras.optimizers.Adam(lr=0.001))
        self.model = model

In [31]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train') # time execution : 7 minutes
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/021 | Loss 0.0021 | Win/lose count 2.0/2.0 (0.0)
Epoch 001/021 | Loss 0.0025 | Win/lose count 2.5/2.0 (0.5)
Epoch 002/021 | Loss 0.0034 | Win/lose count 2.5/1.0 (1.5)
Epoch 003/021 | Loss 0.0007 | Win/lose count 7.0/4.0 (3.0)
Epoch 004/021 | Loss 0.0623 | Win/lose count 5.0/3.0 (2.0)
Epoch 005/021 | Loss 0.0072 | Win/lose count 2.5/5.0 (-2.5)
Epoch 006/021 | Loss 0.0015 | Win/lose count 2.5/7.0 (-4.5)
Epoch 007/021 | Loss 0.0007 | Win/lose count 3.5/2.0 (1.5)
Epoch 008/021 | Loss 0.0027 | Win/lose count 3.5/0 (3.5)
Epoch 009/021 | Loss 0.0034 | Win/lose count 3.0/3.0 (0.0)
Epoch 010/021 | Loss 0.0047 | Win/lose count 4.0/1.0 (3.0)
Epoch 011/021 | Loss 0.0009 | Win/lose count 1.5/1.0 (0.5)
Epoch 012/021 | Loss 0.0036 | Win/lose count 10.0/7.0 (3.0)
Epoch 013/021 | Loss 0.0108 | Win/lose count 5.0/5.0 (0.0)
Epoch 014/021 | Loss 0.0040 | Win/lose count 6.5/1.0 (5.5)
Epoch 015/021 | Loss 0.0579 | Win/lose count 3.5/2.0 (1.5)
Epoch 016/021 | Loss 0.0021 | Win/lose count 6.5/0 (6.5

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [32]:
env = Environment(grid_size=size, max_time=T,temperature=0.3) 
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32) 
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 128)# changed bs from 32 to 128
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json') # changed agent_cnn to agent_fc
# agent_cnn.load(name_weights='fc_trainmodel.h5', name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 11.5/1.0. Average score (10.5)
Win/lose count 3.5/0. Average score (7.0)
Win/lose count 4.0/2.0. Average score (5.333333333333333)
Win/lose count 8.0/3.0. Average score (5.25)
Win/lose count 5.5/0. Average score (5.3)
Win/lose count 11.0/1.0. Average score (6.083333333333333)
Win/lose count 7.0/2.0. Average score (5.928571428571429)
Win/lose count 3.5/2.0. Average score (5.375)
Win/lose count 8.0/3.0. Average score (5.333333333333333)
Win/lose count 12.0/2.0. Average score (5.8)
Win/lose count 8.0/4.0. Average score (5.636363636363637)
Final score: 5.636363636363637
Test of the FC
Win/lose count 6.5/0. Average score (6.5)
Win/lose count 9.0/1.0. Average score (7.25)
Win/lose count 9.0/4.0. Average score (6.5)
Win/lose count 7.5/3.0. Average score (6.0)
Win/lose count 2.5/2.0. Average score (4.9)
Win/lose count 4.5/0. Average score (4.833333333333333)
Win/lose count 7.0/2.0. Average score (4.857142857142857)
Win/lose count 8.0/3.0. Average score (4.875)
Wi

In [35]:
HTML(display_videos('cnn_test10.mp4'))

In [37]:
HTML(display_videos('fc_test10.mp4'))


By looking at the following results (20 epochs for training, 10 for test), one can observe that the CNN works better for all the different temperature values. A higher temperature implies a better performance because when the temperature is high, there are more cheese in the grid and less poisonous cells (malus[bonus>0]=0 in the Environment class). One issue that can be observed is the lack of exploration.


temperature=0.1:

DQN training algorithm using a CNN: 2.86

DQN training algorithm using a cascade of FC layers: 1.18


temperature=0.3:

DQN training algorithm using a CNN: 5.64

DQN training algorithm using a cascade of FC layers: 4.37


temperature=0.5:

DQN training algorithm using a CNN: 17.0

DQN training algorithm using a cascade of FC layers: 6.91


temperature=0.7:

DQN training algorithm using a CNN: 23.91

DQN training algorithm using a cascade of FC layers: 8.91


temperature=0.9:

DQN training algorithm using a CNN: 39.82

DQN training algorithm using a cascade of FC layers: 13.82



***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0


    for e in range(epoch): # 40 epochs
        # decreasing epsilon-greedy exploration
        eps = 1 - e*0.1
        if e < 10: # 1 to 0.1 with a 0.1 step until epoch 10, then 0.1
          agent.set_epsilon(eps)
        else:
          agent.set_epsilon(0.1)


        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')


        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256 
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        self.position[-2:, :] = -1
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        # self.position[-2:, :] = -1 # rather self.position[:,-2:]
        self.position[:,-2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0: # right
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1: # left
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2: # down
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3: # up
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1

        # reward = self.board[self.x, self.y]
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]

        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        # self.position[-2:, :] = -1 # rather self.position[:,-2:]
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        self.malus_position = np.zeros((self.grid_size,self.grid_size))
        self.malus_position[self.x, self.y] = 0.1
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state



## use those samples of code:
#In train explore:
# state, reward, game_over = env.act(action, train=True)

## In Environment exploring:
# You will have to change n_state to 3 because you will use one more layer!
# reward = 0
# if train:
#     reward = -self.malus_position[self.x, self.y]
# self.malus_position[self.x, self.y] = 0.1

# reward = reward + self.board[self.x, self.y]
# # 3 "feature" states instead of 2
# state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
#                                 self.board.reshape(self.grid_size, self.grid_size,1),
#                         self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

In [29]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, 41, prefix='cnn_train_explore') # time execution : 
HTML(display_videos('cnn_train_explore40.mp4'))

Epoch 000/041 | Loss 0.0124 | Win/lose count 12.0/25.50000000000002 (-13.500000000000021)
Epoch 001/041 | Loss 0.0046 | Win/lose count 9.5/28.000000000000078 (-18.500000000000078)
Epoch 002/041 | Loss 0.0069 | Win/lose count 5.5/20.200000000000028 (-14.700000000000028)
Epoch 003/041 | Loss 0.0075 | Win/lose count 9.0/25.400000000000052 (-16.400000000000052)
Epoch 004/041 | Loss 0.0079 | Win/lose count 15.0/24.900000000000013 (-9.900000000000013)
Epoch 005/041 | Loss 0.0064 | Win/lose count 13.0/21.300000000000058 (-8.300000000000058)
Epoch 006/041 | Loss 0.0071 | Win/lose count 11.5/18.700000000000003 (-7.200000000000003)
Epoch 007/041 | Loss 0.0067 | Win/lose count 8.5/18.399999999999995 (-9.899999999999995)
Epoch 008/041 | Loss 0.0383 | Win/lose count 13.0/16.799999999999976 (-3.799999999999976)
Epoch 009/041 | Loss 0.0064 | Win/lose count 14.5/14.199999999999969 (0.3000000000000309)
Epoch 010/041 | Loss 0.0061 | Win/lose count 10.5/16.29999999999996 (-5.799999999999962)
Epoch 011/04

In [30]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 21.5/6.0. Average score (15.5)
Win/lose count 17.0/0. Average score (16.25)
Win/lose count 21.0/6.0. Average score (15.833333333333334)
Win/lose count 17.5/2.0. Average score (15.75)
Win/lose count 25.0/9.0. Average score (15.8)
Win/lose count 26.0/6.0. Average score (16.5)
Win/lose count 19.0/4.0. Average score (16.285714285714285)
Win/lose count 14.0/3.0. Average score (15.625)
Win/lose count 23.0/5.0. Average score (15.88888888888889)
Win/lose count 16.0/2.0. Average score (15.7)
Win/lose count 23.0/4.0. Average score (16.0)
Final score: 16.0


## Final results

Parameters used:

temperature of 0.3

20 epochs for training DQN_FC and DQN_CNN (without encouraging exploration) 

40 epochs for training DQN_CNN_explore (encouraging exploration)

10 epochs for the evaluation


### Final score: 

DQN_FC: **4.37**

DQN_CNN: **5.64**

DQN_CNN_explore: **16**

 

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***